In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tickers = {
    'HDFCBANK.NS': 'HDFC Bank',
    'ICICIBANK.NS': 'ICICI Bank',
    'INFY.NS': 'Infosys',
    'TCS.NS': 'TCS',
    'HINDUNILVR.NS': 'HUL',
    'ITC.NS': 'ITC',
    'TATAMOTORS.NS': 'Tata Motors',
    'MARUTI.NS': 'Maruti',
    'SUNPHARMA.NS': 'Sun Pharma',
    'CIPLA.NS': 'Cipla',
    'LT.NS': 'L&T',
    'RELIANCE.NS': 'Reliance',
    'NTPC.NS': 'NTPC',
    'TITAN.NS': 'Titan',
    'INDIGO.NS': 'IndiGo'
}

# Benchmarking NIFTY 50
benchmark = '^NSEI'

# Date range is from Jan 1st, 2024 to present
start_date = '2024-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

data = yf.download(list(tickers.keys()) + [benchmark], start=start_date, end=end_date)['Close']

In [ ]:
# Renaming columns to company names
data = data.rename(columns=tickers)
data.columns = [col if col in tickers.values() else 'NIFTY 50' for col in data.columns]

# Drop rows with any missing values
data.dropna(inplace=True)

data.head()

In [ ]:
# Calculate daily percentage returns
returns = data.pct_change().dropna()

# Equal-weighted portfolio return
weights = np.array([1/len(tickers)] * len(tickers))
portfolio_returns = returns.iloc[:, :-1].dot(weights)  # exclude NIFTY 50

# Combining into one DataFrame
returns['Portfolio'] = portfolio_returns
returns['NIFTY 50'] = returns['NIFTY 50']

returns.head()

In [ ]:
# Cumulative return (starting from 100)
cumulative = (1 + returns[['Portfolio', 'NIFTY 50']]).cumprod() * 100

cumulative.plot(figsize=(12,6), title='Portfolio vs NIFTY 50 – Cumulative Returns')
plt.ylabel('Value (Indexed to 100)')
plt.grid(True)
plt.show()

In [ ]:
# Volatility (standard deviation of returns)
volatility = returns.std() * 100

# Max drawdown calculation
rolling_max = cumulative.cummax()
drawdown = cumulative / rolling_max - 1
max_drawdown = drawdown.min() * 100

# Summary
print("Volatility (%):")
print(volatility.round(2))

print("\nMax Drawdown (%):")
print(max_drawdown.round(2))

In [ ]:
# Export daily returns & drawdown to CSV
returns.to_csv("daily_returns.csv")
drawdown.to_csv("drawdown.csv")
cumulative.to_csv("cumulative_returns.csv")